In [ ]:
!pip install torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from google.colab import drive
drive.mount('/content/drive')

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist

In [2]:
#SCRIPT PARA ENTRENAR EL MODELO

with open('drive/My Drive/PRETRAINING/Torrestube.txt', 'r', encoding='utf-8') as f:
    texto = f.read()

# hiperparámetros
tamaño_lote = 32 # secuencias independientes para procesar en paralelo
tamaño_bloque = 64 # longitud máxima del contexto para predicciones
max_iters = 10000
intervalo_evaluacion = 200
tasa_aprendizaje = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_capa = 4
dropout = 0.0
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu' # usar GPU si está disponible
# ------------



# Caracteres únicos que ocurren en este texto
caracteres = sorted(list(set(texto)))
tamaño_vocabulario = len(caracteres)

# Crea un mapeo de caracteres a enteros
caracteres_a_enteros = { ch:i for i,ch in enumerate(caracteres) }
enteros_a_caracteres = { i:ch for i,ch in enumerate(caracteres) }
codificar = lambda s: [caracteres_a_enteros[c] for c in s] # codificador: tomar una cadena, producir una lista de enteros
decodificar = lambda l: ''.join([enteros_a_caracteres[i] for i in l]) # decodificador: tomar una lista de enteros, producir una cadena

# Divisiones de entrenamiento y prueba
datos = torch.tensor(codificar(texto), dtype=torch.long)
n = int(0.9*len(datos)) # el primer 90% será entrenamiento, el resto validación
datos_entrenamiento = datos[:n]
datos_validacion = datos[n:]

torch.manual_seed(5483)

# carga de datos
def obtener_lote(division):
    # generar un pequeño lote de datos de entradas x y objetivos y
    datos = datos_entrenamiento if division == 'train' else datos_validacion
    ix = torch.randint(len(datos) - tamaño_bloque, (tamaño_lote,))
    x = torch.stack([datos[i:i+tamaño_bloque] for i in ix])
    y = torch.stack([datos[i+1:i+tamaño_bloque+1] for i in ix])
    x, y = x.to(dispositivo), y.to(dispositivo)
    return x, y

@torch.no_grad()
def estimar_perdida():
    out = {}
    modelo.eval()
    for division in ['train', 'val']:
        perdidas = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = obtener_lote(division)
            logits, perdida = modelo(X, Y)
            perdidas[k] = perdida.item()
        out[division] = perdidas.mean()
    modelo.train()
    return out

class Cabeza(nn.Module):
    """ una cabeza de autoatención """

    def __init__(self, tamaño_cabeza):
        super().__init__()
        self.clave = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.consulta = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.valor = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(tamaño_bloque, tamaño_bloque)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.clave(x)   # (B,T,C)
        q = self.consulta(x) # (B,T,C)
        # calcular puntajes de atención ("afinidades")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # realizar la agregación ponderada de los valores
        v = self.valor(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class AtencionMultiCabeza(nn.Module):
    """ múltiples cabezas de autoatención en paralelo """

    def __init__(self, num_cabezas, tamaño_cabeza):
        super().__init__()
        self.cabezas = nn.ModuleList([Cabeza(tamaño_cabeza) for _ in range(num_cabezas)])
        self.proy = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.cabezas], dim=-1)
        out = self.dropout(self.proy(out))
        return out

class FeedForward(nn.Module):
    """ una capa lineal seguida de una no-linealidad """

    def __init__(self, n_embd):
        super().__init__()
        self.red = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.red(x)

class Bloque(nn.Module):
    """ Bloque transformador: comunicación seguida de cálculo """

    def __init__(self, n_embd, n_cabeza):
        # n_embd: dimensión del embedding, n_cabeza: el número de cabezas que nos gustaría
        super().__init__()
        tamaño_cabeza = n_embd // n_cabeza
        self.atn = AtencionMultiCabeza(n_cabeza, tamaño_cabeza)
        self.avance = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.atn(self.ln1(x))
        x = x + self.avance(self.ln2(x))
        return x

# modelo bigrama simple
class ModeloBigrama(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token lee directamente los logits para el siguiente token desde una tabla de búsqueda
        self.tabla_embedding_token = nn.Embedding(tamaño_vocabulario, n_embd)
        self.tabla_embedding_posición = nn.Embedding(tamaño_bloque, n_embd)
        self.bloques = nn.Sequential(*[Bloque(n_embd, n_cabeza=n_head) for _ in range(n_capa)])
        self.ln_f = nn.LayerNorm(n_embd) # capa final de normalización
        self.cabeza_lm = nn.Linear(n_embd, tamaño_vocabulario)

    def forward(self, idx, objetivos=None):
        B, T = idx.shape

        # idx y objetivos son ambos tensores (B, T) de enteros
        emb_tok = self.tabla_embedding_token(idx) # (B,T,C)
        emb_pos = self.tabla_embedding_posición(torch.arange(T, device=dispositivo)) # (T,C)
        x = emb_tok + emb_pos # (B,T,C)
        x = self.bloques(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.cabeza_lm(x) # (B,T,tamaño_vocabulario)

        if objetivos is None:
            perdida = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            objetivos = objetivos.view(B*T)
            perdida = F.cross_entropy(logits, objetivos)

        return logits, perdida

    def generar(self, idx, máx_nuevos_tokens):
        # idx es un arreglo (B, T) de índices en el contexto actual
        for _ in range(máx_nuevos_tokens):
            # recortar idx a los últimos tamaño_bloque tokens
            idx_cond = idx[:, -tamaño_bloque:]
            # obtener las predicciones
            logits, perdida = self(idx_cond)
            # enfocarse solo en el último paso de tiempo
            logits = logits[:, -1, :] # se convierte en (B, C)
            # aplicar softmax para obtener probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # muestrear de la distribución
            idx_siguiente = torch.multinomial(probs, num_samples=1) # (B, 1)
            # agregar el índice muestreado a la secuencia en ejecución
            idx = torch.cat((idx, idx_siguiente), dim=1) # (B, T+1)
        return idx

# imprimir el número de parámetros en el modelo
print(sum(p.numel() for p in m.parameters())/1e6, 'M parámetros')

modelo = ModeloBigrama()
m = modelo.to(dispositivo)
# crear un optimizador PyTorch
optimizador = torch.optim.AdamW(modelo.parameters(), lr=tasa_aprendizaje)

for iteracion in range(max_iters):

    # cada cierto tiempo evaluar la pérdida en los conjuntos de entrenamiento y validación
    if iteracion % intervalo_evaluacion == 0 or iteracion == max_iters - 1:
        perdidas = estimar_perdida()
        print(f"paso {iteracion}: pérdida de entrenamiento {perdidas['train']:.4f}, pérdida de validación {perdidas['val']:.4f}")

    # obtener un lote de datos
    xb, yb = obtener_lote('train')

    # evaluar la pérdida
    logits, perdida = modelo(xb, yb)
    optimizador.zero_grad(set_to_none=True)
    perdida.backward()
    optimizador.step()

torch.save(modelo.state_dict(), 'drive/My Drive/PRETRAINING/torrestmodelo.pth')

contexto = torch.zeros((1, 1), dtype=torch.long, device=dispositivo)
print(decodificar(m.generar(contexto, máx_nuevos_tokens=100)[0].tolist()))


FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/PRETRAINING/Torrestube.txt'